In [1]:
import numpy as np
import cv2 as cv
import os
import json
import matplotlib.pyplot as plt

%load_ext autoreload
%autoreload 2

In [ ]:
data_path = os.path.expanduser("~/LunarAutonomyChallenge/output/hazard_1")

i = 401
I1 = cv.imread(os.path.join(data_path, "front_left", f"{i}.png"), cv.IMREAD_GRAYSCALE)
plt.imshow(I1, cmap="gray")
plt.show()

In [ ]:
i = 701
I1 = cv.imread(os.path.join(data_path, "front_left", f"{i}.png"), cv.IMREAD_GRAYSCALE)
plt.imshow(I1, cmap="gray")
plt.show()